In [3]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### GDP growth contributions table

In [2]:
# Settings for GDP table
slist = ['A191RL', 'DPCERY', 'DDURRY', 'DNDGRY', 'DSERRY', 'A006RY',
         'A008RY', 'A011RY', 'A014RY', 'A822RY', 'A823RY', 'A829RY', 
         'A019RY', 'A020RY', 'A021RY']

n = {'A191RL': '\cbox{red!95!black} & \\textbf{Gross Domestic Product}',
     'DPCERY': '\cbox{yellow!80!orange} & \hspace{2mm} Consumer Spending',
     'DDURRY': '& \hspace{4mm} Durable Goods',
     'DNDGRY': '& \hspace{4mm} Non-Durable Goods ',
     'DSERRY': '& \hspace{4mm} Services ',
     'A006RY': '\cbox{blue!70!black} & \hspace{2mm} Gross Investment',
     'A011RY': '& \hspace{4mm} Residential ',
     'A008RY': '& \hspace{4mm} Non-Residential ',
     'A014RY': '& \hspace{4mm} Change in Inventories ',
     'A019RY': '\cbox{green!60!black} & \hspace{2mm} Net Exports ',
     'A020RY': '& \hspace{4mm} Exports ',
     'A021RY': '& \hspace{4mm} Imports ',
     'A822RY': '\cbox{cyan!50!white} & \hspace{2mm} Government ',
     'A823RY': '& \hspace{4mm} Federal ',
     'A829RY': '& \hspace{4mm} State and Local '}


va = pd.read_csv(data_dir / 'gdpvafull.csv', parse_dates=['date']).set_index('date')

sl2 = ['GoodsTTU', 'Manufacturing', 'Construction', 'Retail trade', 'FIRE', 
       'Oth_Serv', 'Educational services, health care, and social assistance',
       'Professional and business services', 'Information', 'Government']

n2 = {'GoodsTTU': '\cbox{purple!70!blue} & \hspace{2mm} Goods and TTU ',
      'Manufacturing': '& \hspace{4mm} Manufacturing ',
      'Construction': '& \hspace{4mm} Construction ',
      'Retail trade': '& \hspace{4mm} Retail Trade ',
      'FIRE': '\cbox{red!90!white} & \hspace{2mm} FIRE+ ',
      'Oth_Serv': '\cbox{blue!90!white} & \hspace{2mm} Other Services ',
      'Educational services, health care, and social assistance': '& \hspace{4mm} Education \& Healthcare ',
      'Professional and business services': '& \hspace{4mm} Professional \& Business',
      'Information': '& \hspace{4mm} Information ',
      'Government': '\cbox{orange!80!white} & \hspace{2mm} Government '}

jb = pd.read_csv(data_dir / 'gdpjobs.csv', parse_dates=['date']).set_index('date') 

n3 = {'pop_contr': '\cbox{lime!90!green} & \hspace{2mm} Population ',
      'epop_contr': '\cbox{green!30!teal!80!black} & \hspace{2mm} Employment Rate ',
      'hours_contr': '\cbox{blue} & \hspace{2mm} Average Hours',
      'prod': '\cbox{cyan!55!white} & \hspace{2mm} Productivity '}

sl3 = ['pop_contr', 'epop_contr', 'hours_contr', 'prod']

gd = pd.read_csv(data_dir / 'gdi.csv', parse_dates=['date']).set_index('date')

n4 = {'A261RC': '& \\textbf{Gross Domestic Income} ',
      'A4002C': '\cbox{magenta!90!blue} & \hspace{2mm} Labor ',
      'W271RC': '\cbox{yellow!60!orange} & \hspace{2mm} Profit ',
      'A262RC': '\cbox{teal!60!white} & \hspace{2mm} Depreciation ',
      'indirect': '\cbox{violet} & \hspace{2mm} Indirect Taxes '}

sl4 = ['A261RC', 'A4002C', 'W271RC', 'A262RC', 'indirect']

In [3]:
df = nipa_df(retrieve_table('T10502')['Data'], slist).sort_index()
#nipa_series_codes(retrieve_table('T10502'))

In [4]:
# GDP Table (please consider revising!!)
f = data_dir / 'gdptable.tex'
os.remove(f)

date1 = {}
date = {}
data = {s: {} for s in list(df.keys())}

blankrow = '& & & & & & & & & & \\\\'

with open(f, 'a') as table:
    
    for s in list(df.keys()):
        data[s]['name'] = n[s]

    for i in range(1, 6):
        dt = df['A191RL'].index[-i]
        q = dt.quarter
        y = dt.year
        date1[i] = dt
        date[i] = f'{y} Q{q}'
        for s in list(df.keys()):
            if s == 'A191RL':
                data[s][i] = f'{df[s].iloc[-i]:.1f}'
            else:
                data[s][i] = f'{df[s].iloc[-i]:.2f}'

    for ma in [13, 41, 121]:
        for s in list(df.keys()):
            if s == 'A191RL':
                data[s][ma] = f'{df[s].iloc[-ma:].mean():.1f}'
            else:
                data[s][ma] = f'{df[s].iloc[-ma:].mean():.2f}'

    daterow = (f'& & {date[1]} & {date[2]} & {date[3]} & {date[4]} & {date[5]}'
                ' & & 3-year & 10-year & 30-year \\\\')
    table.write(daterow + '\n')
    print(daterow)

    for s in slist:
        r = (f'{n[s]} & {data[s][1]} & {data[s][2]} & {data[s][3]} & '
             f'{data[s][4]} & {data[s][5]} & & {data[s][13]} & '
             f' {data[s][41]} & {data[s][121]} \\\\')

        print(r)
        table.write(r + '\n')

    print(blankrow)
    table.write(blankrow + '\n')

    for s in list(va.keys()):
        data[s] = {}
        for i in range(1, 6):
            if date1[i] in va.index:
                data[s][i] = f'{va[s].loc[date1[i]]:.2f}'
            else:
                data[s][i] = '--'

    for ma in [13, 41, 121]:
        for s in list(va.keys()):
            data[s][ma] = f'{va[s].iloc[-ma:].mean():.2f}'

    for s in sl2:
        r = (f'{n2[s]} & {data[s][1]} & {data[s][2]} & {data[s][3]} & '
             f'{data[s][4]} & {data[s][5]} & & {data[s][13]} & '
             f' {data[s][41]} & {data[s][121]} \\\\')
        print(r)
        table.write(r + '\n')

    print(blankrow)
    table.write(blankrow + '\n')

    for s in list(jb.keys()):
        data[s] = {}
        for i in range(1, 6):
            if date1[i] in jb.index:
                data[s][i] = f'{jb[s].loc[date1[i]]:.2f}'
            else:
                data[s][i] = '--'

    for ma in [13, 41, 121]:
        for s in list(jb.keys()):
            data[s][ma] = f'{jb[s].iloc[-ma:].mean():.2f}'

    for s in sl3:
        r = (f'{n3[s]} & {data[s][1]} & {data[s][2]} & {data[s][3]} & '
             f'{data[s][4]} & {data[s][5]} & & {data[s][13]} & '
             f' {data[s][41]} & {data[s][121]} \\\\')
        print(r)
        table.write(r + '\n')

    print(blankrow)
    table.write(blankrow)

    for s in list(gd.keys()):
        data[s] = {}
        if s == 'A261RC':
            for i in range(1, 6):
                if date1[i] in gd.index:
                    data[s][i] = f'{gd[s].loc[date1[i]]:.1f}'
                else:
                    data[s][i] = '--'
        else:
            for i in range(1, 6):
                if date1[i] in gd.index:
                    data[s][i] = f'{gd[s].loc[date1[i]]:.2f}'
                else:
                    data[s][i] = '--'

    for ma in [13, 41, 121]:
        for s in list(gd.keys()):
            if s == 'A261RC':
                data[s][ma] = f'{gd[s].iloc[-ma:].mean():.1f}'
            else:
                data[s][ma] = f'{gd[s].iloc[-ma:].mean():.2f}'

    for s in sl4:
        r = (f'{n4[s]} & {data[s][1]} & {data[s][2]} & {data[s][3]} & '
             f'{data[s][4]} & {data[s][5]} & & {data[s][13]} & '
             f' {data[s][41]} & {data[s][121]} \\\\')
        if s == 'indirect':
            r = r.replace('\\\\', '')
        print(r)
        table.write(r + '\n')

& & 2023 Q3 & 2023 Q2 & 2023 Q1 & 2022 Q4 & 2022 Q3 & & 3-year & 10-year & 30-year \\
\cbox{red!95!black} & \textbf{Gross Domestic Product} & 4.9 & 2.1 & 2.2 & 2.6 & 2.7 & & 5.6 &  2.6 & 2.6 \\
\cbox{yellow!80!orange} & \hspace{2mm} Consumer Spending & 2.69 & 0.55 & 2.54 & 0.79 & 1.05 & & 4.35 &  1.90 & 1.87 \\
& \hspace{4mm} Durable Goods & 0.60 & -0.03 & 1.07 & -0.08 & 0.08 & & 0.77 &  0.49 & 0.48 \\
& \hspace{4mm} Non-Durable Goods  & 0.48 & 0.14 & 0.07 & 0.07 & -0.26 & & 0.67 &  0.46 & 0.38 \\
& \hspace{4mm} Services  & 1.62 & 0.44 & 1.40 & 0.80 & 1.23 & & 2.92 &  0.95 & 1.02 \\
\cbox{blue!70!black} & \hspace{2mm} Gross Investment & 1.47 & 0.90 & -1.69 & 0.62 & -1.45 & & 1.60 &  0.71 & 0.69 \\
& \hspace{4mm} Non-Residential  & 0.00 & 0.98 & 0.76 & 0.24 & 0.62 & & 0.87 &  0.55 & 0.59 \\
& \hspace{4mm} Residential  & 0.15 & -0.09 & -0.22 & -1.23 & -1.41 & & -0.00 &  0.07 & 0.04 \\
& \hspace{4mm} Change in Inventories  & 1.32 & 0.00 & -2.22 & 1.61 & -0.66 & & 0.73 &  0.09 & 0.07 \\
\c